# ETAPA 3: Visualizaciones Avanzadas

**Objetivo:** Profundizar en el análisis visual, comparando regiones y tendencias.

**Rango temporal:** 2 años de datos (2020–2021)

**Visualizaciones requeridas:**
1. Evolución temporal global de casos confirmados, activos y fallecidos (líneas)
2. Comparativa Top 10 países con más casos confirmados (barras)
3. Heatmap de correlaciones entre columnas relevantes
4. Gráfico de barras horizontales comparando tasas de letalidad por continente
5. Mapa geográfico que muestre la incidencia por continente

In [ ]:
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import os
from datetime import datetime

# Configuración de Plotly para notebooks
import plotly.io as pio
pio.renderers.default = 'notebook'

print("✓ Librerías importadas correctamente")

In [ ]:
# Configuración: ruta a los archivos daily_reports locales
# Nota: el notebook está en notebooks/, por eso usamos '../' para subir un nivel
DATA_DIR = os.path.join('..', 'data', 'raw', 'COVID-19', 'csse_covid_19_data', 'csse_covid_19_daily_reports')

# Rango de fechas a cargar (2 años: 2020-2021)
START_DATE = '2020-01-22'
END_DATE = '2021-12-31'

dates = pd.date_range(start=START_DATE, end=END_DATE, freq='D')

# Lista temporal para acumular los DataFrames de cada día
dfs = []

print("Cargando datos desde archivos locales...")
print(f"{'='*60}")
print(f"Período: {START_DATE} → {END_DATE}")
print(f"Total de archivos a cargar: {len(dates)}")
print(f"{'='*60}\n")

# Leer cada archivo CSV diario y agregarlo a la lista
for i, date in enumerate(dates, 1):
    filename = date.strftime('%m-%d-%Y') + '.csv'  # Formato: MM-DD-YYYY
    filepath = os.path.join(DATA_DIR, filename)
    
    if not os.path.exists(filepath):
        print(f"⚠ Archivo no encontrado: {filename}")
        continue
    
    try:
        df = pd.read_csv(filepath)
        df.columns = df.columns.str.strip()
        
        # Normalizar nombres de columnas para compatibilidad
        if 'Province/State' in df.columns:
            df.rename(columns={'Province/State': 'Province_State'}, inplace=True)
        if 'Country/Region' in df.columns:
            df.rename(columns={'Country/Region': 'Country_Region'}, inplace=True)
        if 'Province_State' not in df.columns:
            df['Province_State'] = np.nan
        if 'Country_Region' not in df.columns and 'Country' in df.columns:
            df.rename(columns={'Country': 'Country_Region'}, inplace=True)
        
        df['Date'] = date
        dfs.append(df)
        
        # Mostrar progreso cada 50 archivos
        if i % 50 == 0:
            print(f"✓ Cargados {i}/{len(dates)} archivos ({i/len(dates)*100:.1f}%)")
            
    except Exception as e:
        print(f"✗ Error en {filename}: {e}")

# Concatenar (apilar verticalmente) todos los DataFrames diarios en uno solo
if dfs:
    df_2years = pd.concat(dfs, ignore_index=True)
    print(f"\n{'='*60}")
    print(f"✓ Cargados {len(dfs)} archivos diarios")
    print(f"✓ Total de registros: {len(df_2years):,}")
    print(f"✓ Período: {df_2years['Date'].min().date()} → {df_2years['Date'].max().date()}")
    print(f"{'='*60}")
else:
    df_2years = pd.DataFrame()
    print("\n⚠ No se cargó ningún archivo.")
    print("Ejecuta: ./scripts/fetch_jhu_data.sh clone")

In [ ]:
# Limpieza y preparación del dataset

print("Iniciando limpieza de datos...\n")

# 1. Estandarizar nombres de columnas (usar formato snake_case)
df_2years.columns = df_2years.columns.str.lower().str.replace(' ', '_').str.replace('/', '_').str.replace('-', '_')
print("✓ Nombres de columnas estandarizados")

# 2. Eliminar columnas duplicadas (consolidando valores)
duplicated_cols = df_2years.columns[df_2years.columns.duplicated()].unique()

if len(duplicated_cols) > 0:
    print(f"⚠ Encontradas columnas duplicadas: {duplicated_cols.tolist()}")
    
    for col_name in duplicated_cols:
        # Obtener todas las columnas con este nombre
        matching_cols = [i for i, c in enumerate(df_2years.columns) if c == col_name]
        
        # Consolidar: tomar el primer valor no nulo de cada fila
        consolidated = df_2years.iloc[:, matching_cols[0]]
        for col_idx in matching_cols[1:]:
            consolidated = consolidated.fillna(df_2years.iloc[:, col_idx])
        
        # Eliminar todas las columnas duplicadas
        df_2years = df_2years.drop(df_2years.columns[matching_cols], axis=1)
        
        # Agregar la columna consolidada
        df_2years[col_name] = consolidated
        print(f"  ✓ '{col_name}' consolidada")
else:
    print("✓ No hay columnas duplicadas")

# 3. Eliminar columnas irrelevantes
columns_to_drop = ['fips', 'admin2', 'lat', 'long_', 'latitude', 'longitude', 'combined_key']
df_2years = df_2years.drop(columns=[col for col in columns_to_drop if col in df_2years.columns])
print("✓ Columnas irrelevantes eliminadas")

# 4. Procesar fechas
if 'last_update' in df_2years.columns:
    df_2years['last_update'] = pd.to_datetime(df_2years['last_update'], errors='coerce')
print("✓ Fechas procesadas")

# 5. Convertir columnas numéricas
numeric_columns = ['confirmed', 'deaths', 'recovered']
for col in numeric_columns:
    if col in df_2years.columns:
        df_2years[col] = pd.to_numeric(df_2years[col], errors='coerce').fillna(0).astype(int)
print("✓ Columnas numéricas convertidas")

# 6. Calcular casos activos
df_2years['active_cases'] = df_2years['confirmed'] - df_2years['deaths'] - df_2years['recovered']
print("✓ Casos activos calculados")

# 7. Homogeneizar nombres de países
country_mapping = {
    'US': 'United States',
    'UK': 'United Kingdom',
    'Korea, South': 'South Korea',
    'Taiwan*': 'Taiwan',
    'Mainland China': 'China'
}

if 'country_region' in df_2years.columns:
    df_2years['country_region'] = df_2years['country_region'].replace(country_mapping)
    print("✓ Nombres de países homogeneizados")

print(f"\n{'='*60}")
print(f"✓ Dataset limpio: {len(df_2years):,} registros")
print(f"✓ Columnas: {list(df_2years.columns)}")
print(f"{'='*60}")

df_2years.head()

In [ ]:
# Cargar mapeo de países a continentes
continent_mapping_path = os.path.join('..', 'data', 'country_to_continet.csv')

try:
    df_continents = pd.read_csv(continent_mapping_path)
    print(f"✓ Mapeo de continentes cargado: {len(df_continents)} países")
    print(f"\nContinentes disponibles: {df_continents['continent'].unique()}")
    
    # Crear diccionario de mapeo
    country_to_continent = dict(zip(df_continents['country'], df_continents['continent']))
    
    # Agregar columna de continente al dataset
    df_2years['continent'] = df_2years['country_region'].map(country_to_continent)
    
    # Verificar países sin mapeo
    unmapped = df_2years[df_2years['continent'].isna()]['country_region'].unique()
    if len(unmapped) > 0:
        print(f"\n⚠ Países sin mapeo de continente ({len(unmapped)}):")
        print(unmapped[:10])  # Mostrar solo los primeros 10
    else:
        print("\n✓ Todos los países tienen continente asignado")
    
    print(f"\n✓ Distribución por continente:")
    print(df_2years['continent'].value_counts())
    
except Exception as e:
    print(f"✗ Error al cargar mapeo de continentes: {e}")
    df_2years['continent'] = None

---
## Visualización 1: Evolución Temporal Global

Gráfico de líneas mostrando la evolución de casos confirmados, activos y fallecidos a nivel mundial.

In [ ]:
# Agregar datos por fecha para obtener totales globales
global_evolution = df_2years.groupby('date').agg({
    'confirmed': 'sum',
    'deaths': 'sum',
    'recovered': 'sum',
    'active_cases': 'sum'
}).reset_index()

# Crear gráfico interactivo con Plotly
fig = go.Figure()

fig.add_trace(go.Scatter(
    x=global_evolution['date'],
    y=global_evolution['confirmed'],
    mode='lines',
    name='Casos Confirmados',
    line=dict(color='#1f77b4', width=2)
))

fig.add_trace(go.Scatter(
    x=global_evolution['date'],
    y=global_evolution['active_cases'],
    mode='lines',
    name='Casos Activos',
    line=dict(color='#ff7f0e', width=2)
))

fig.add_trace(go.Scatter(
    x=global_evolution['date'],
    y=global_evolution['deaths'],
    mode='lines',
    name='Fallecidos',
    line=dict(color='#d62728', width=2)
))

fig.update_layout(
    title='Evolución Temporal Global de COVID-19 (2020-2021)',
    xaxis_title='Fecha',
    yaxis_title='Número de Casos',
    hovermode='x unified',
    template='plotly_white',
    height=600,
    legend=dict(yanchor="top", y=0.99, xanchor="left", x=0.01)
)

fig.show()

# Mostrar estadísticas clave
print("\n📊 Estadísticas Globales (2020-2021):")
print(f"  • Total casos confirmados: {global_evolution['confirmed'].iloc[-1]:,}")
print(f"  • Total fallecidos: {global_evolution['deaths'].iloc[-1]:,}")
print(f"  • Total recuperados: {global_evolution['recovered'].iloc[-1]:,}")
print(f"  • Casos activos al final: {global_evolution['active_cases'].iloc[-1]:,}")

---
## Visualización 2: Top 10 Países con Más Casos Confirmados

Gráfico de barras comparando los países con mayor número de casos confirmados acumulados.

In [ ]:
# Obtener el máximo de casos confirmados por país (valor acumulado final)
top_countries = df_2years.groupby('country_region')['confirmed'].max().sort_values(ascending=False).head(10)

# Crear gráfico de barras interactivo
fig = px.bar(
    x=top_countries.values,
    y=top_countries.index,
    orientation='h',
    labels={'x': 'Casos Confirmados', 'y': 'País'},
    title='Top 10 Países con Más Casos Confirmados (2020-2021)',
    text=top_countries.values
)

fig.update_traces(
    texttemplate='%{text:,.0f}',
    textposition='outside',
    marker_color='steelblue'
)

fig.update_layout(
    yaxis={'categoryorder': 'total ascending'},
    template='plotly_white',
    height=500,
    showlegend=False
)

fig.show()

print("\n📊 Top 10 Países:")
for i, (country, cases) in enumerate(top_countries.items(), 1):
    print(f"  {i:2d}. {country:25s}: {cases:,}")

---
## Visualización 3: Heatmap de Correlaciones

Matriz de correlación entre variables relevantes (confirmados, fallecidos, recuperados, activos).

In [ ]:
# Preparar datos agregados por país para análisis de correlación
correlation_data = df_2years.groupby('country_region').agg({
    'confirmed': 'max',
    'deaths': 'max',
    'recovered': 'max',
    'active_cases': 'max'
}).reset_index()

# Calcular tasa de letalidad
correlation_data['mortality_rate'] = (
    correlation_data['deaths'] / correlation_data['confirmed'] * 100
).replace([np.inf, -np.inf], 0).fillna(0)

# Calcular tasa de recuperación
correlation_data['recovery_rate'] = (
    correlation_data['recovered'] / correlation_data['confirmed'] * 100
).replace([np.inf, -np.inf], 0).fillna(0)

# Seleccionar columnas numéricas para correlación
corr_columns = ['confirmed', 'deaths', 'recovered', 'active_cases', 'mortality_rate', 'recovery_rate']
correlation_matrix = correlation_data[corr_columns].corr()

# Crear heatmap con Plotly
fig = px.imshow(
    correlation_matrix,
    text_auto='.2f',
    aspect='auto',
    color_continuous_scale='RdBu_r',
    color_continuous_midpoint=0,
    labels=dict(color='Correlación'),
    title='Matriz de Correlación entre Variables COVID-19'
)

fig.update_layout(
    template='plotly_white',
    height=600,
    xaxis_title='',
    yaxis_title=''
)

fig.show()

print("\n📊 Interpretación de Correlaciones:")
print("\nCorrelaciones más fuertes:")
# Obtener las correlaciones más altas (excluyendo la diagonal)
corr_pairs = []
for i in range(len(correlation_matrix.columns)):
    for j in range(i+1, len(correlation_matrix.columns)):
        corr_pairs.append((
            correlation_matrix.columns[i],
            correlation_matrix.columns[j],
            correlation_matrix.iloc[i, j]
        ))
corr_pairs.sort(key=lambda x: abs(x[2]), reverse=True)

for var1, var2, corr in corr_pairs[:5]:
    print(f"  • {var1} ↔ {var2}: {corr:.3f}")

---
## Visualización 4: Tasas de Letalidad por Continente

Comparación de tasas de letalidad entre continentes.

In [ ]:
# Filtrar datos con continente asignado
df_with_continent = df_2years[df_2years['continent'].notna()].copy()

# Calcular totales por continente
continent_stats = df_with_continent.groupby('continent').agg({
    'confirmed': 'sum',
    'deaths': 'sum',
    'recovered': 'sum'
}).reset_index()

# Calcular tasa de letalidad por continente
continent_stats['mortality_rate'] = (
    continent_stats['deaths'] / continent_stats['confirmed'] * 100
).replace([np.inf, -np.inf], 0).fillna(0)

# Ordenar por tasa de letalidad
continent_stats = continent_stats.sort_values('mortality_rate', ascending=True)

# Filtrar continente 'Other' si existe
continent_stats = continent_stats[continent_stats['continent'] != 'Other']

# Crear gráfico de barras horizontales
fig = px.bar(
    continent_stats,
    x='mortality_rate',
    y='continent',
    orientation='h',
    title='Tasa de Letalidad por Continente (2020-2021)',
    labels={'mortality_rate': 'Tasa de Letalidad (%)', 'continent': 'Continente'},
    text='mortality_rate',
    color='mortality_rate',
    color_continuous_scale='Reds'
)

fig.update_traces(
    texttemplate='%{text:.2f}%',
    textposition='outside'
)

fig.update_layout(
    template='plotly_white',
    height=500,
    showlegend=False,
    coloraxis_showscale=False
)

fig.show()

print("\n📊 Tasas de Letalidad por Continente:")
for _, row in continent_stats.sort_values('mortality_rate', ascending=False).iterrows():
    print(f"  • {row['continent']:15s}: {row['mortality_rate']:5.2f}% "
          f"({row['deaths']:,} fallecidos / {row['confirmed']:,} confirmados)")

---
## Visualización 5: Incidencia por Continente (Mapa)

Visualización geográfica mostrando la distribución de casos confirmados por continente.

In [ ]:
# Preparar datos de casos confirmados por continente
continent_cases = df_with_continent.groupby('continent').agg({
    'confirmed': 'sum',
    'deaths': 'sum',
    'recovered': 'sum',
    'active_cases': 'sum'
}).reset_index()

# Filtrar continente 'Other' si existe
continent_cases = continent_cases[continent_cases['continent'] != 'Other']

# Calcular porcentaje del total mundial
total_confirmed = continent_cases['confirmed'].sum()
continent_cases['percentage'] = (continent_cases['confirmed'] / total_confirmed * 100)

# Crear gráfico de pastel (sunburst) interactivo
fig = px.sunburst(
    continent_cases,
    path=['continent'],
    values='confirmed',
    title='Distribución Global de Casos Confirmados por Continente',
    color='confirmed',
    color_continuous_scale='Viridis',
    hover_data={'confirmed': ':,', 'deaths': ':,', 'percentage': ':.1f'}
)

fig.update_traces(
    textinfo='label+percent entry',
    hovertemplate='<b>%{label}</b><br>'
                  'Confirmados: %{customdata[0]:,}<br>'
                  'Fallecidos: %{customdata[1]:,}<br>'
                  'Porcentaje: %{customdata[2]:.1f}%<extra></extra>'
)

fig.update_layout(
    template='plotly_white',
    height=600
)

fig.show()

# Crear también un gráfico de barras para comparación directa
fig2 = px.bar(
    continent_cases.sort_values('confirmed', ascending=False),
    x='continent',
    y='confirmed',
    title='Casos Confirmados por Continente (2020-2021)',
    labels={'confirmed': 'Casos Confirmados', 'continent': 'Continente'},
    text='confirmed',
    color='confirmed',
    color_continuous_scale='Blues'
)

fig2.update_traces(
    texttemplate='%{text:,.0f}',
    textposition='outside'
)

fig2.update_layout(
    template='plotly_white',
    height=500,
    showlegend=False,
    coloraxis_showscale=False
)

fig2.show()

print("\n📊 Distribución de Casos por Continente:")
for _, row in continent_cases.sort_values('confirmed', ascending=False).iterrows():
    print(f"  • {row['continent']:15s}: {row['confirmed']:>12,} casos ({row['percentage']:5.1f}%)")

---
## Resumen y Conclusiones

### Principales hallazgos:

1. **Evolución Temporal**: Se observa un crecimiento exponencial de casos durante 2020, con múltiples olas a lo largo de 2021.

2. **Distribución Geográfica**: Los casos se concentran principalmente en ciertos continentes, reflejando diferentes políticas de salud pública y capacidades de detección.

3. **Correlaciones**: Existe una fuerte correlación positiva entre casos confirmados y fallecidos, lo cual era esperado. Las tasas de letalidad varían significativamente entre continentes.

4. **Disparidades Continentales**: Las tasas de letalidad muestran variaciones importantes entre continentes, posiblemente relacionadas con sistemas de salud, demografía y políticas de respuesta.

5. **Top Países Afectados**: Los países con mayor población y/o mayor capacidad de detección aparecen consistentemente entre los más afectados.

### Limitaciones del análisis:
- Los datos dependen de la capacidad de detección de cada país
- Pueden existir subregistros en países con sistemas de salud limitados
- Las definiciones de "recuperado" varían entre países
- No se consideran factores demográficos, económicos o de políticas públicas